# Limestone Layer Analysis — Notebook

This notebook mirrors the CLI workflow in `script/analyze_limestone.py`.

It will:
- Load the input CSV.
- Run the interpolation and filling (IDW + Trend + Anisotropic residuals).
- Save outputs to `output/` with timestamped filenames.
- Preview the filled dataset and cross‑validation summary.

Notes:
- Uses only the repository code; pandas previews are optional (fallback to stdlib `csv` if pandas is not installed).
- Set site location and depth datum as appropriate to your project.


In [ ]:
# Paths & metadata
import os

# Project root inferred from this notebook's location
here = os.path.abspath(os.path.dirname(os.getcwd())) if os.path.basename(os.getcwd()) == 'notebook' else os.path.abspath(os.getcwd())
project_root = here
csv_filename = 'limestone_layers_phase_1_input.csv'
input_csv = os.path.join(project_root, 'input', csv_filename)
output_dir = os.path.join(project_root, 'output')

# Optional metadata (edit as needed)
location = None  # e.g., 'Project XYZ, Zone A'
datum = None     # e.g., 'RL to MSL'

input_csv, output_dir, location, datum


In [ ]:
# Run analysis
import importlib.util, sys
from pathlib import Path

# Ensure repository root is in sys.path so we can import script.analyze_limestone
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from script.analyze_limestone import run_analysis

out_path = run_analysis(input_csv=input_csv, output_dir=output_dir, location=location, datum=datum)
out_path


In [ ]:
# Preview outputs: filled CSV (head)
filled_csv = out_path
cv_csv = os.path.join(output_dir, 'cv_summary.csv')
report_md = os.path.join(output_dir, 'report.md')

print('Filled CSV :', filled_csv)
print('CV Summary:', cv_csv)
print('Report MD :', report_md)

try:
    import pandas as pd
    display(pd.read_csv(filled_csv).head(10))
except Exception as e:
    import csv as _csv
    print('(pandas unavailable, using csv module)')
    with open(filled_csv, newline='', encoding='utf-8') as f:
        r = _csv.DictReader(f)
        rows = []
        for i, row in enumerate(r):
            rows.append(row)
            if i >= 9:
                break
    for row in rows:
        print(row)


In [ ]:
# Preview CV summary (sorted by RMSE ascending)
try:
    import pandas as pd
    cv_df = pd.read_csv(cv_csv)
    display(cv_df.sort_values('RMSE', ascending=True))
except Exception as e:
    import csv as _csv
    print('(pandas unavailable, using csv module)')
    with open(cv_csv, newline='', encoding='utf-8') as f:
        r = _csv.DictReader(f)
        for row in r:
            print(row)


In [ ]:
# Optional: simple scatter maps if pandas and matplotlib are available
try:
    import pandas as pd
    import matplotlib.pyplot as plt
    plt.style.use('fivethirtyeight')
    df = pd.read_csv(filled_csv)
    # Plot X vs Y colored by sounding_beg_limestone
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    sc1 = ax[0].scatter(df['easting_coord_x'], df['northing_coord_y'], c=df['sounding_beg_limestone'], cmap='viridis', s=25)
    ax[0].set_title('Sounding begin limestone (m)')
    ax[0].set_xlabel('Easting X')
    ax[0].set_ylabel('Northing Y')
    plt.colorbar(sc1, ax=ax[0])
    # Plot X vs Y colored by limestone_thickness
    sc2 = ax[1].scatter(df['easting_coord_x'], df['northing_coord_y'], c=df['limestone_thickness'], cmap='plasma', s=25)
    ax[1].set_title('Limestone thickness (m)')
    ax[1].set_xlabel('Easting X')
    ax[1].set_ylabel('Northing Y')
    plt.colorbar(sc2, ax=ax[1])
    plt.tight_layout()
    plt.show()
except Exception as e:
    print('Skipping plots (dependency missing):', e)
